# CustomFactor Class Documentation

*Disclaimer: This documentation was generated by AI and may require human revision for accuracy and completeness.*

## Overview

The `CustomFactor` class in GTSAM is a specialized type of factor used in factor graphs, which allows users to define custom error functions and Jacobians. This class is particularly useful when the standard factors provided by GTSAM do not meet specific application needs, allowing for greater flexibility and customization in defining the behavior of the factor.

## Key Functionalities

### Custom Error Function

The `CustomFactor` class allows users to define a custom error function. This is a critical feature as it enables the modeling of specific constraints or measurements that are not covered by existing GTSAM factors. The error function typically represents the difference between predicted and observed measurements and is central to the optimization process in factor graphs.

### Jacobian Calculation

In addition to the error function, users can define custom Jacobians. The Jacobian is the matrix of all first-order partial derivatives of a vector-valued function. In the context of `CustomFactor`, it represents the sensitivity of the error function with respect to the variables involved. This is crucial for optimization algorithms, which rely on gradient information to find the minimum of the error function.

### Integration with Factor Graphs

`CustomFactor` seamlessly integrates with GTSAM's factor graph framework. It can be added to a factor graph just like any other factor, participating in the graph optimization process. This integration ensures that custom factors can be used alongside standard factors, providing a flexible and powerful tool for solving complex estimation problems.

## Mathematical Formulation

### Error Function

The error function $e(\mathbf{x})$ is defined by the user and represents the discrepancy between the observed and predicted measurements. It is a vector-valued function:

$$ e(\mathbf{x}) = h(\mathbf{x}) - \mathbf{z} $$

where $h(\mathbf{x})$ is the predicted measurement based on the current estimate of the variables $\mathbf{x}$, and $\mathbf{z}$ is the observed measurement.

### Jacobian

The Jacobian matrix $J$ of the error function with respect to the variables $\mathbf{x}$ is given by:

$$ J = \frac{\partial e}{\partial \mathbf{x}} $$

This matrix is used in the optimization process to update the estimates of the variables in a direction that reduces the error.

## Usage

To use `CustomFactor`, users must:

1. Define the custom error function that models the specific measurement or constraint.
2. Implement the calculation of the Jacobian matrix for the error function.
3. Add the `CustomFactor` to a factor graph, specifying the keys of the variables it depends on.

By following these steps, users can leverage the flexibility of `CustomFactor` to incorporate custom measurements and constraints into their factor graph models, enhancing the capability of GTSAM to solve a wide range of estimation problems.